<a href="https://colab.research.google.com/github/rajasriramoju/CS269-Attenuating-Bias/blob/main/Attentuating_word_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

​​[Paper](https://arxiv.org/pdf/1901.07656.pdf)
 
Parts of coode being used from: https://github.com/sunipa/Attenuating-Bias-in-Word-Vec


In [1]:
import gensim
import json
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr

## Part I: Pretained Embeddings

The paper has looked into several word embeddings:
- GloVe embedding trained on Wiki dump
- Word2Vec embedding trained on Google News

**We are using the latter embedding for this tutorial.**

Given a pretained embedding, the paper evaluates its bias using several metrics:
- WEAT
- EQT
- ECT

**Here, we are going to only use ECT metrics to be concise.**

In [2]:
import gensim.downloader
model = gensim.downloader.load('word2vec-google-news-300')
print('Pre-trained model has been loaded.')

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Pre-trained model has been loaded.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def processList(l):
	for i in range(len(l)):
		l[i] = l[i].strip().lower()
	return l

def meanList(l):
	vec= [0] * 300
	for i in range(len(l)):
		vec = vec + model.get_vector(l[i])
	return vec/float(len(l))

In [5]:
occupations = open('/content/drive/Shareddrives/CS269-Midterm/Attenuating-Bias-in-Word-Vec-master/wordList.txt','r')
occupations = processList(occupations.readlines())
print(occupations)

['detective', 'ambassador', 'coach', 'officer', 'epidemiologist', 'rabbi', 'ballplayer', 'secretary', 'actress', 'manager', 'scientist', 'cardiologist', 'actor', 'industrialist', 'welder', 'biologist', 'undersecretary', 'captain', 'economist', 'politician', 'baron', 'pollster', 'environmentalist', 'photographer', 'mediator', 'character', 'housewife', 'jeweler', 'physicist', 'hitman', 'geologist', 'painter', 'employee', 'stockbroker', 'footballer', 'tycoon', 'dad', 'patrolman', 'chancellor', 'advocate', 'bureaucrat', 'strategist', 'pathologist', 'psychologist', 'campaigner', 'magistrate', 'judge', 'illustrator', 'surgeon', 'nurse', 'missionary', 'stylist', 'solicitor', 'scholar', 'naturalist', 'artist', 'mathematician', 'businesswoman', 'investigator', 'curator', 'soloist', 'servant', 'broadcaster', 'fisherman', 'landlord', 'housekeeper', 'crooner', 'archaeologist', 'teenager', 'councilman', 'attorney', 'choreographer', 'principal', 'parishioner', 'therapist', 'administrator', 'skipper'

### Using the top-10 most common male and female names

In [6]:
maleNames = open('/content/drive/Shareddrives/CS269-Midterm/Attenuating-Bias-in-Word-Vec-master/maleNames.txt','r') 
m = meanList(processList(maleNames.readlines())) 
m.shape

(300,)

In [7]:
femaleNames =open('/content/drive/Shareddrives/CS269-Midterm/Attenuating-Bias-in-Word-Vec-master/femaleNames.txt','r') 
s = meanList(processList(femaleNames.readlines()))
s.shape

(300,)

### ECT Score Implementation

In [8]:
def ect(mean1, mean2, wordlist):
    sim1=[0]*len(wordlist)
    sim2=[0]*len(wordlist)

    for i in range(0,len(wordlist)):
        sim1[i] = 1 - cosine(mean1, model.get_vector(wordlist[i]))
        sim2[i] = 1 - cosine(mean2, model.get_vector(wordlist[i]))
    return spearmanr(sim1, sim2)

In [9]:
ect(m,s,occupations)

SpearmanrResult(correlation=0.7438807204756531, pvalue=5.611070211556043e-33)

In [10]:
import numpy as np

In [76]:
# john, mary, cinematographer
male_occ = np.dot(model.get_vector('john'), model.get_vector('cinematographer'))
print(male_occ)

1.000562


In [77]:
# john, mary, cinematographer
female_occ = np.dot(model.get_vector('mary'), model.get_vector('cinematographer'))
print(female_occ)

0.26169828


In [71]:
male_names = ['john','william','george','louis','michael','tony','andrew','daniel','scott','jackson']
female_names = ['mary','victoria','carolina','maria','anne','kelly','marie','anna','sarah','jane']



Neutralization should ideally bring the Spearman coefficient towards 1.

## Part II: Debiasing the embedding

The paper has looked into several methods of debiasing that includes
- subtraction
- projection
- hard debiasing
- their own solution that avoids crowd-sourcing

We will look into the last solution as it is the paper's main contribution.



### Step 1: Computing mean of equality set

---



In [42]:
equality_set = open('/content/drive/Shareddrives/CS269-Midterm/Attenuating-Bias-in-Word-Vec-master/equality_sets.txt', 'r')
mw_list = []
sw_list = []
for pair in equality_set:
    mw, sw = pair.split(' ')[0], pair.split(' ')[1]
    mw_list.append(mw)
    sw_list.append(sw)


In [43]:
print(len(mw_list))

67


In [44]:
mu = [0] * 300
for i in range(len(mw_list)):
    
    mu = mu + model.get_vector(mw_list[i])
    mu = mu + model.get_vector(sw_list[i])


In [45]:
mu = mu / (len(mw_list)*2)

### Step 2: Compute the gender directional vector

In [46]:
v_b = (s - m)/np.linalg.norm(s-m)

### Step 3: Compute the inherent bias $\beta$

In [47]:
def compute_inherent_bias(word):
    return np.dot(word, v_b) - np.dot(mu, v_b)

### Step 4: Compute the redidual orthogonal component

In [48]:
def compute_orthor(word):
    return word - np.dot(word, v_b) * v_b

### Step 5: Compiling everything up


In [49]:
def compute_debiased_vector(word, f):
    return mu + compute_orthor(word) + compute_inherent_bias(word) * f * v_b

### Step 6: Experimenting on different functions



In [50]:
def f1(sigma, word):
    return sigma**2 / (np.linalg.norm(compute_orthor(word) + 1.))**2

In [51]:
def f2(sigma, word):
    n = np.linalg.norm(compute_orthor(word))
    return np.exp(-n**2/sigma**2)

In [52]:
def f3(sigma, word):
    n = np.linalg.norm(compute_orthor(word))
    return max(0, sigma/2*n)

In [53]:
def ect_debiased(mean1, mean2, wordlist, func):
    sim1=[0]*len(wordlist)
    sim2=[0]*len(wordlist)

    for i in range(0,len(wordlist)):
        debiased_vector = compute_debiased_vector(model.get_vector(wordlist[i]), func(1., model.get_vector(wordlist[i])))
        sim1[i] = 1 - cosine(mean1, debiased_vector)
        sim2[i] = 1 - cosine(mean2, debiased_vector)
    return spearmanr(sim1, sim2)

In [54]:
ect_debiased(m,s,occupations,f1)

SpearmanrResult(correlation=0.9979896478179859, pvalue=3.318968210457974e-215)

In [55]:
ect_debiased(m,s,occupations,f2)

SpearmanrResult(correlation=0.9979196867389803, pvalue=6.948905672415539e-214)

In [56]:
ect_debiased(m,s,occupations,f3)

SpearmanrResult(correlation=0.4338008722500409, pvalue=1.1769026641293876e-09)

In [78]:
# john, mary, cinematographer
debiased_vec = compute_debiased_vector(model.get_vector('cinematographer'), f1(1., model.get_vector('cinematographer')))
male_occ = np.dot(model.get_vector('john'), debiased_vec)
print(male_occ)

2.431401215696626


In [79]:
# john, mary, cinematographer
female_occ = np.dot(model.get_vector('mary'), debiased_vec)
print(female_occ)

1.7626235745686012
